<a href="https://colab.research.google.com/github/divyansh351/ML-Bootcamp-WOC/blob/main/2LayerNeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/MyDrive/cyberlabs/emnist_data2
train_path = '/content/drive/MyDrive/cyberlabs/emnist_data2/emnist-letters-train.csv'
test_path = '/content/drive/MyDrive/cyberlabs/emnist_data2/emnist-letters-test.csv'
df_train = pd.read_csv(train_path, header=None)
df_test = pd.read_csv(test_path, header=None)

/content/drive/MyDrive/cyberlabs/emnist_data2


In [3]:
print('Loading Data ...\n')
X=df_train.drop([0], axis=1)
X_train=X.to_numpy().astype('float64')
X_train = X_train/255.0
m,n=X_train.shape
X_train = X_train.T
Y_train=df_train[0].to_numpy().astype('int').reshape(1, m)
m1,n1=Y_train.shape
print(X_train,'\nShape -', m, n,'\n')
print(Y_train,'\nShape -', m1, n1)
Y_train = Y_train - 1

Loading Data ...

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] 
Shape - 64783 784 

[[ 1  1  1 ... 19 19 19]] 
Shape - 1 64783


In [4]:
print('Loading Data ...\n')
X=df_test.drop([0], axis=1)
X_test=X.to_numpy().astype('float64')
X_test = X_test/255.0
mt,nt=X_test.shape
X_test = X_test.T
Y_test=df_test[0].to_numpy().astype('int').reshape(1, mt)
mt1,nt1=Y_test.shape
print(X_test,'\nShape -', mt, nt,'\n')
print(Y_test, '\nShape -', mt1, nt1)
Y_test = Y_test - 1

Loading Data ...

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]] 
Shape - 14800 784 

[[ 1  1  1 ... 19 19 19]] 
Shape - 1 14800


In [13]:
def init_params():
    W1 = np.random.rand(25, 784) - 0.5
    b1 = np.random.rand(25, 1) - 0.5
    W2 = np.random.rand(25, 25) - 0.5
    b2 = np.random.rand(25, 1) - 0.5
    W3 = np.random.rand(19, 25) - 0.5
    b3 = np.random.rand(19, 1) - 0.5
    return W1, b1, W2, b2, W3, b3

def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A
    
def forward_prop(W1, b1, W2, b2, W3, b3, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    Z3 = W3.dot(A2) + b3
    A3 = softmax(Z3)
    return Z1, A1, Z2, A2, Z3, A3

def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, Z3, A3, W1, W2, W3, X, Y):
    one_hot_Y = one_hot(Y)
    dZ3 = A3 - one_hot_Y
    dW3 = 1 / m * dZ3.dot(A2.T)
    db3 = 1 / m * np.sum(dZ3)
    dZ2 = W3.T.dot(dZ3) * ReLU_deriv(Z2)
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2, dW3, db3

def update_params(W1, b1, W2, b2, W3, b3, dW1, db1, dW2, db2, dW3, db3, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2
    W3 = W3 - alpha * dW3
    b3 = b3 - alpha * db3    
    return W1, b1, W2, b2, W3, b3

In [22]:
def get_predictions(A3):
    return np.argmax(A3, 0)

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2, W3, b3 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2, Z3, A3 = forward_prop(W1, b1, W2, b2, W3, b3, X)
        dW1, db1, dW2, db2, dW3, db3 = backward_prop(Z1, A1, Z2, A2, Z3, A3, W1, W2, W3, X, Y)
        W1, b1, W2, b2, W3, b3 = update_params(W1, b1, W2, b2, W3, b3, dW1, db1, dW2, db2, dW3, db3, alpha)
        if i % 50 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2, W3, b3

In [24]:
W1, b1, W2, b2, W3, b3 = gradient_descent(X_train, Y_train, 0.005, 501)

Iteration:  0
[17 17 17 ... 17  6 24] [[ 0  0  0 ... 18 18 18]]
0.03584273652038343
Iteration:  50
[17 17 17 ... 17  6 24] [[ 0  0  0 ... 18 18 18]]
0.03466958924409182
Iteration:  100
[17 17 17 ... 23  6 24] [[ 0  0  0 ... 18 18 18]]
0.03371254804501181
Iteration:  150
[17 17 17 ... 23  6 17] [[ 0  0  0 ... 18 18 18]]
0.03267832610407051
Iteration:  200
[17 17 17 ... 23  6 17] [[ 0  0  0 ... 18 18 18]]
0.03144343423428986
Iteration:  250
[17 17 17 ... 23  6 17] [[ 0  0  0 ... 18 18 18]]
0.030640754518932434
Iteration:  300
[17 17 17 ... 23  6 17] [[ 0  0  0 ... 18 18 18]]
0.029236065017056945
Iteration:  350
[17 17 17 ... 23  6 17] [[ 0  0  0 ... 18 18 18]]
0.02776963092169242
Iteration:  400
[17 17 17 ... 23  6 17] [[ 0  0  0 ... 18 18 18]]
0.02551595325934273
Iteration:  450
[17 17 17 ... 23  6 23] [[ 0  0  0 ... 18 18 18]]
0.023941466125372398
Iteration:  500
[16 17 17 ... 23  6 23] [[ 0  0  0 ... 18 18 18]]
0.02313878641001497
